# Separate data into different positions and do feature selection in order to train different models

In [45]:
import os
from src.features.data_engineering import get_merged_seasons_data

In [46]:
data = get_merged_seasons_data()

## Separate data into different positions

In [47]:
data_gk = data[data['position'] == 'GK']
data_def = data[data['position'] == 'DEF']
data_mid = data[data['position'] == 'MID']
data_fwd = data[data['position'] == 'FWD']

In [48]:
print('GK len:', len(data_gk))
print('DEF len:', len(data_def))
print('MID len:', len(data_mid))
print('FWD len:', len(data_fwd))

GK len: 14504
DEF len: 45353
MID len: 55126
FWD len: 18615


## Feature selection for different positions

In [49]:
# preview columns from data
data.columns

Index(['name', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity',
       'element', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index',
       'influence', 'kickoff_time', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'position', 'season'],
      dtype='object')

Remember about so called 'target features':
- (['name', 'GW', 'element', 'total_points_next_gameweek', 'season'])

And about features dropped during preprocessing:
- (['fixture', 'kickoff_time', 'opponent_team', 'round', 'transfers_balance', 'was_home'])

### Select features to drop

In [50]:
data_gk.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,14504.000000,14504.000000,14504.000000,14504.000000,14504.000000,14504.000000,14504.000000,14504.000000,14504.000000,14504.000000,...,14499.000000,14499.000000,14504.000000,14504.000000,1.450400e+04,14504.000000,1.450400e+04,14504.000000,14504.000000,14504.000000
mean,0.000896,0.083149,5.535990,0.085494,0.081833,329.473318,192.275510,0.414782,0.000069,0.686025,...,1.234223,1.496862,0.007033,1.104592,4.984468e+02,7382.392581,6.883946e+03,44.969388,0.011514,20.256481
std,0.029926,0.423872,9.355694,0.279625,0.939281,187.487637,106.365156,0.938966,0.008303,1.308831,...,1.205719,1.306483,0.267694,2.286974,2.526136e+04,22281.962485,2.017316e+04,4.820750,0.106688,11.388042
min,0.000000,0.000000,-7.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-3.000000,-1.035061e+06,0.000000,0.000000e+00,39.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,165.000000,102.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-1.089000e+03,50.000000,1.410000e+02,40.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,329.000000,194.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-7.600000e+01,337.000000,8.965000e+02,44.000000,0.000000,20.000000
75%,0.000000,0.000000,11.000000,0.000000,0.000000,495.000000,281.000000,0.000000,0.000000,1.000000,...,2.000000,2.000000,0.000000,1.000000,7.500000e+01,4468.750000,6.557500e+03,48.000000,0.000000,29.000000
max,1.000000,3.000000,54.000000,1.000000,30.000000,708.000000,380.000000,9.000000,1.000000,10.300000,...,9.000000,9.000000,18.000000,17.000000,4.190200e+05,459020.000000,1.039664e+06,63.000000,1.000000,47.000000


In [51]:
drop_features_gk = ['assists', 'creativity', 'goals_scored', 'ict_index', 'own_goals', 'penalties_missed', 'red_cards', 'threat', 'yellow_cards']

In [52]:
data_def.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,45353.000000,45353.000000,45353.000000,45353.000000,45353.000000,45353.000000,45353.000000,45353.000000,45353.000000,45353.000000,...,45328.000000,45328.000000,45353.000000,45353.000000,4.535300e+04,45353.000000,4.535300e+04,45353.000000,45353.000000,45353.000000
mean,0.025842,0.101118,7.015920,0.112209,3.417029,299.456464,190.515534,0.550636,0.016846,1.441501,...,1.236785,1.500441,2.954909,1.300928,6.462538e+02,9168.141093,8.521231e+03,46.786034,0.064384,20.053337
std,0.167980,0.459563,9.905467,0.315626,8.498592,177.287403,107.499723,1.028436,0.131407,2.269528,...,1.209492,1.305887,8.143801,2.512014,3.700206e+04,31944.028488,2.767015e+04,6.209284,0.245438,11.480167
min,0.000000,0.000000,-12.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-7.000000,-1.012731e+06,0.000000,0.000000e+00,37.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,146.000000,98.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-1.451000e+03,55.000000,1.810000e+02,43.000000,0.000000,10.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,297.000000,192.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-8.300000e+01,442.000000,1.216000e+03,45.000000,0.000000,20.000000
75%,0.000000,0.000000,13.000000,0.000000,1.300000,443.000000,281.000000,1.000000,0.000000,2.400000,...,2.000000,2.000000,2.000000,1.000000,1.590000e+02,4163.000000,6.120000e+03,50.000000,0.000000,29.000000
max,3.000000,3.000000,75.000000,1.000000,109.500000,713.000000,380.000000,9.000000,2.000000,22.800000,...,9.000000,9.000000,152.000000,24.000000,9.350820e+05,941215.000000,1.025064e+06,85.000000,1.000000,47.000000


In [53]:
drop_features_def = ['own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves']

In [54]:
data_mid.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,55126.000000,55126.000000,55126.000000,55126.000000,55126.000000,55126.000000,55126.000000,55126.000000,55126.000000,55126.000000,...,55104.000000,55104.000000,55126.000000,55126.000000,5.512600e+04,5.512600e+04,5.512600e+04,55126.000000,55126.000000,55126.000000
mean,0.056833,0.094148,5.767968,0.109894,7.213297,305.888728,191.580379,0.491202,0.054185,2.085564,...,1.241870,1.514845,6.663643,1.406451,9.089597e+02,1.061898e+04,9.709814e+03,54.208540,0.060461,20.169884
std,0.253803,0.463828,9.253016,0.312760,13.457418,177.974122,107.540642,0.945799,0.251080,3.482679,...,1.215104,1.318324,14.457536,2.469952,5.475263e+04,4.719898e+04,3.984044e+04,13.250668,0.238342,11.489364
min,0.000000,0.000000,-14.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-2.000000,-1.857821e+06,0.000000e+00,0.000000e+00,40.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,155.000000,100.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-8.960000e+02,2.800000e+01,7.100000e+01,45.000000,0.000000,10.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,299.000000,194.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-5.000000e+01,2.130000e+02,5.835000e+02,50.000000,0.000000,20.000000
75%,0.000000,0.000000,9.000000,0.000000,10.900000,453.000000,282.000000,1.000000,0.000000,3.100000,...,2.000000,2.000000,6.000000,2.000000,1.000000e+01,2.383750e+03,4.388750e+03,58.000000,0.000000,29.000000
max,4.000000,3.000000,93.000000,1.000000,170.900000,714.000000,380.000000,9.000000,4.000000,35.800000,...,9.000000,9.000000,181.000000,29.000000,1.907229e+06,1.991731e+06,1.872898e+06,136.000000,1.000000,47.000000


In [55]:
drop_features_mid = ['own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves']

In [56]:
data_fwd.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,...,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,GW
count,18615.000000,18615.000000,18615.000000,18615.000000,18615.000000,18615.000000,18615.000000,18615.000000,18615.000000,18615.000000,...,18608.000000,18608.000000,18615.000000,18615.000000,1.861500e+04,1.861500e+04,1.861500e+04,18615.000000,18615.000000,18615.000000
mean,0.052216,0.163309,4.775020,0.088155,4.623717,318.231104,192.584851,0.433253,0.110771,2.237534,...,1.223828,1.504837,11.604781,1.466882,1.755410e+03,2.016722e+04,1.841057e+04,60.730164,0.039860,20.259683
std,0.243454,0.620014,11.043327,0.283527,9.093162,184.143104,107.460852,0.889658,0.368351,3.971370,...,1.201224,1.291700,21.515047,2.713212,7.829593e+04,6.685945e+04,5.747027e+04,17.124864,0.195636,11.463868
min,0.000000,0.000000,-19.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-2.000000,-1.734284e+06,0.000000e+00,0.000000e+00,42.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,152.000000,101.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,-2.258500e+03,8.500000e+01,2.420000e+02,49.000000,0.000000,11.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,326.000000,195.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,-1.000000e+02,8.210000e+02,1.862000e+03,56.000000,0.000000,20.000000
75%,0.000000,0.000000,4.000000,0.000000,3.900000,476.000000,283.000000,1.000000,0.000000,3.000000,...,2.000000,2.000000,17.000000,2.000000,1.115000e+02,8.211000e+03,1.183950e+04,65.000000,0.000000,29.000000
max,4.000000,3.000000,114.000000,1.000000,92.000000,711.000000,380.000000,9.000000,4.000000,31.100000,...,9.000000,9.000000,199.000000,24.000000,1.983733e+06,2.104464e+06,1.782662e+06,132.000000,1.000000,47.000000


In [57]:
drop_features_fwd = ['clean_sheets', 'goals_conceded', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves']

### Drop features and save data to intermediate folder

In [58]:
data_gk = data_gk.drop(drop_features_gk, axis=1)
data_def = data_def.drop(drop_features_def, axis=1)
data_mid = data_mid.drop(drop_features_mid, axis=1)
data_fwd = data_fwd.drop(drop_features_fwd, axis=1)

In [60]:
data_intermediate_path = os.path.dirname(os.getcwd()) + '\\data\\intermediate\\'

data_gk.to_csv(data_intermediate_path + 'separate/gk.csv', index=False)
data_def.to_csv(data_intermediate_path + 'separate/def.csv', index=False)
data_mid.to_csv(data_intermediate_path + 'separate/mid.csv', index=False)
data_fwd.to_csv(data_intermediate_path + 'separate/fwd.csv', index=False)